In [1]:
import pandas as pd
import numpy as np
import sklearn.tree as tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os.path
from os import path
from sklearn.model_selection import KFold
from multiprocessing import Process, Manager, SimpleQueue
import multiprocessing
import shutil
import sys
import time
from sklearn.metrics import accuracy_score
from rules import *
from vote import *
from save_amie_data import *
from precision import *
from amie import *

root = "./../../DT_Tree_Constant_GT/"
cv = 3

colors_to_print = ["red",  "blue", "orange",  "violet", "darkgrey", "darkred", "darkblue", "darkorange", "darkviolet", "grey"]

# 1° Load data

In [2]:
data = pd.read_csv("./../Data/miniloan-decisions-100K.csv")

In [3]:
data

,name,creditScore,income,loanAmount,monthDuration,rate,approval,yearlyReimbursement
0,John Doe,736,113243,783440,162,0.030005,false,70648
1,John Doe,527,224186,1787385,183,0.054819,false,173197
2,John Doe,460,39954,1733494,75,0.055183,false,328563
3,John Doe,751,219998,1350004,72,0.043400,false,255970
4,John Doe,684,72470,1210944,160,0.037772,false,115742
...,...,...,...,...,...,...,...,...
99995,John Doe,366,83851,1718571,168,0.053737,false,174928
99996,John Doe,395,184259,1599473,90,0.040564,false,247703
99997,John Doe,674,242620,1099790,260,0.040825,false,76557
99998,John Doe,420,180957,1645870,203,0.049558,false,143897


# Ratios

In [4]:
ratio_names = [("income","yearlyReimbursement"), ("income","loanAmount"), ("loanAmount","rate")]

In [5]:
# Takes two features and will create the ratio f_0/f_1
def create_features_ratio(data):
    if data[data.index[1]] == 0:
        return 2147483647
    return np.round(data[data.index[0]]/data[data.index[1]], decimals=2)

In [6]:
for ratio_name in ratio_names:
    data[f"{ratio_name[0]}/{ratio_name[1]}"] = data[[ratio_name[0], ratio_name[1]]].apply(create_features_ratio, axis=1)

In [7]:
data

,name,creditScore,income,loanAmount,monthDuration,rate,approval,yearlyReimbursement,income/yearlyReimbursement,income/loanAmount,loanAmount/rate
0,John Doe,736,113243,783440,162,0.030005,false,70648,1.60,0.14,26110372.80
1,John Doe,527,224186,1787385,183,0.054819,false,173197,1.29,0.13,32605003.59
2,John Doe,460,39954,1733494,75,0.055183,false,328563,0.12,0.02,31413368.70
3,John Doe,751,219998,1350004,72,0.043400,false,255970,0.86,0.16,31106073.81
4,John Doe,684,72470,1210944,160,0.037772,false,115742,0.63,0.06,32059172.14
...,...,...,...,...,...,...,...,...,...,...,...
99995,John Doe,366,83851,1718571,168,0.053737,false,174928,0.48,0.05,31981153.44
99996,John Doe,395,184259,1599473,90,0.040564,false,247703,0.74,0.12,39430859.20
99997,John Doe,674,242620,1099790,260,0.040825,false,76557,3.17,0.22,26938940.88
99998,John Doe,420,180957,1645870,203,0.049558,false,143897,1.26,0.11,33211031.24


# 2° Find the best thresholds

In [8]:
data_raw = data.drop(columns=["name"])
false_data = data_raw[data_raw["approval"] == " false"]
true_data = data_raw[data_raw["approval"] == " true"]

nb_launch_tree = 10

In [9]:
%%time

def returnSplitValue(stringToSplit):
    res = []
    tp = ""
    spliters = [("lt", "< "), ("lte", "<= "), ("gt", "> "), ("gte", ">= ")]
    i=0
    
    while i < len(spliters):
        splited = stringToSplit.split(spliters[i][1]) 
        if len(splited) == 2:
            return  spliters[i][0], float(splited[1])
        i += 1
    print("C'est casse")
    return None 


best_threshold = []
for key in data.drop(columns=["name"]).columns:
    if key != "approval":
        res_accuracy_train = []
        res_accuracy_test = []
        model = None
        print("\t\t"+key)
        for i in range(nb_launch_tree):
            data_equilibrium = true_data.append(false_data.sample(n=len(true_data)))
            X = data_equilibrium[[key]]

            Y = data_equilibrium["approval"]

            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

            model = tree.DecisionTreeClassifier(max_depth=1)
            model.fit(X_train, y_train)    
            
            res_accuracy_train.append(accuracy_score(y_train, model.predict(X_train)))
            res_accuracy_test.append(accuracy_score(y_test, model.predict(X_test)))
        print(f"Train Score : {np.mean(res_accuracy_train)}")
        print(f"Test Score : {np.mean(res_accuracy_test)}")

        if np.mean(res_accuracy_train) > limit_tree and np.mean(res_accuracy_test) > limit_tree:
            best_threshold.append((key,)+ returnSplitValue(tree.export_text(model, spacing=1, feature_names=[key]).split("\n")[0]))
print(best_threshold)

		creditScore
Train Score : 0.5008158270446665
Test Score : 0.4997929606625259
		income
Train Score : 0.6526956285267523
Test Score : 0.6444444444444445
		loanAmount
Train Score : 0.8414236181929431
Test Score : 0.8523809523809524
		monthDuration
Train Score : 0.6155415052008975
Test Score : 0.6139406487232574
		rate
Train Score : 0.5111496362771093
Test Score : 0.5055900621118012
		yearlyReimbursement
Train Score : 0.8980216194166837
Test Score : 0.9049689440993789
		income/yearlyReimbursement
Train Score : 0.9934053980556122
Test Score : 0.9933747412008281
		income/loanAmount
Train Score : 0.9250118974777347
Test Score : 0.9242926155969634
		loanAmount/rate
Train Score : 0.8272486232918621
Test Score : 0.8337474120082816
[('loanAmount', 'lte', 820910.0), ('yearlyReimbursement', 'lte', 66250.5), ('income/yearlyReimbursement', 'lte', 3.33), ('income/loanAmount', 'lte', 0.24), ('loanAmount/rate', 'lte', 17710752.0)]
CPU times: user 2.14 s, sys: 0 ns, total: 2.14 s
Wall time: 2.17 s


# 3° Create the thresholds

Define the thresholds

In [10]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [11]:
# Takes a dataframe and return the values per percents for each feature
def description_per_thresholds(data, percents):
    thresholds_Dic =  {}
    for column in data.columns:
        serie = data[column]
        thresholds_Dic[column] = serie.quantile(percents)
        
    return pd.DataFrame.from_dict(thresholds_Dic)

In [12]:
def GTE_prep_df(value, valueToCompareTo):
    return value >= valueToCompareTo

def LTE_prep_df(value, valueToCompareTo):
    return value <= valueToCompareTo

def GT_prep_df(value, valueToCompareTo):
    return value > valueToCompareTo

def LT_prep_df(value, valueToCompareTo):
    return value < valueToCompareTo

In [13]:
columns = list(data.drop(columns=["name", "approval"]).columns)

booleanDF = pd.DataFrame()

for column, comparator, value in best_threshold:
    if comparator == "lte" or comparator == "gt":
#         booleanDF[column+"-GT-"+str(value)] = data[column].map(lambda x : GT_prep_df(x, value))
        booleanDF[column+"-LTE-"+str(value)] = data[column].map(lambda x : LTE_prep_df(x, value))
    else:
        booleanDF[column+"-GTE-"+str(value)] = data[column].map(lambda x : GTE_prep_df(x, value))
#         booleanDF[column+"-LT-"+str(value)] = data[column].map(lambda x : LT_prep_df(x, value))
    columns.remove(column)

In [14]:
description = description_per_thresholds(data[columns], thresholds)

Transform the data from integer values to boolean values corresponding to GTE or LTE with the different percents.

In [15]:
%%time

for column in columns:
    for i, value in enumerate(thresholds):
        booleanDF[column+"-GTE-"+str(int(value*100))] = data[column].map(lambda x : GTE_prep_df(x, description[column].values[i]))
#         booleanDF[column+"-LTE-"+str(int(value*100))] = data[column].map(lambda x : LTE_prep_df(x, description[column].values[i]))

CPU times: user 5.78 s, sys: 46.9 ms, total: 5.83 s
Wall time: 5.81 s


In [16]:
booleanDF["approval"] = data["approval"].map({' true': True, ' false': False})
booleanDF

,loanAmount-LTE-820910.0,yearlyReimbursement-LTE-66250.5,income/yearlyReimbursement-LTE-3.33,income/loanAmount-LTE-0.24,loanAmount/rate-LTE-17710752.0,creditScore-GTE-25,creditScore-GTE-50,creditScore-GTE-75,income-GTE-25,income-GTE-50,income-GTE-75,monthDuration-GTE-25,monthDuration-GTE-50,monthDuration-GTE-75,rate-GTE-25,rate-GTE-50,rate-GTE-75,approval
0,True,False,True,True,False,True,True,True,True,False,False,True,False,False,False,False,False,False
1,False,False,True,True,False,True,False,False,True,True,False,True,True,False,True,True,True,False
2,False,False,True,True,False,True,False,False,False,False,False,False,False,False,True,True,True,False
3,False,False,True,True,False,True,True,True,True,True,False,False,False,False,True,False,False,False
4,False,False,True,True,False,True,True,False,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,False,False,True,True,False,False,False,False,True,False,False,True,False,False,True,True,True,False
99996,False,False,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False
99997,False,False,True,True,False,True,True,False,True,True,True,True,True,False,True,False,False,False
99998,False,False,True,True,False,False,False,False,True,True,False,True,True,False,True,True,False,False


Create the folder where the data will be stored.

In [17]:
if not path.isdir(root):
    os.mkdir(root)
    os.mkdir(root+"/Save_raw")
else : 
    shutil.rmtree(root)
    os.mkdir(root)
    os.mkdir(root+"/Save_raw")

In [18]:
booleanDF.to_csv(root+"dfSave.csv")

# 4° Save the best data

We store the data in the Amie format. 

In [19]:
save_all_data(root, booleanDF)

## Save in cross validation

In [20]:
save_CV(root, booleanDF, cv)

Train repartition :
 False    52004
True     14662
Name: approval, dtype: int64
Train repartition :
 False    52007
True     14660
Name: approval, dtype: int64
Train repartition :
 False    52081
True     14586
Name: approval, dtype: int64


# 5° Launch Amie

Describe the parameters.

In [2]:
atom_LIST = [2, 3]
minc_LIST = [0, 0.4, 0.8]
minhc_LIST = [0, 0.4, 0.8]

Launch Amie.

In [ ]:
%%time

res_rules_raw = run_amie(atom_LIST, root, cv, const=True)

Process n°0 : Launched
Process n°1 : Launched
Process n°2 : Launched
Process n°3 : Launched
Process n°4 : Launched
Process n°5 : Launched
Process n°6 : Launched
Process n°6 : FinishedProcess n°7 : Launched

Process n°7 : FinishedProcess n°8 : Launched

Process n°8 : FinishedProcess n°9 : Launched

Process n°10 : Launched
Process n°9 : Finished
Process n°10 : FinishedProcess n°11 : Launched

Process n°11 : Finished
Process n°12 : Launched
Process n°12 : Finished
Process n°2 : Finished
Process n°0 : Finished
Process n°4 : Finished


### Save raw results

In [ ]:
for i in res_rules_raw:
    f = open(root+"Save_raw/"+i+".txt", "w")
    f.write(res_rules_raw[i].decode())
    f.close()

### Read raw results

In [ ]:
res_rules_raw = {}
for a in atom_LIST:
    for i in range(cv):
        f = open(root+"Save_raw/"+f"CV={i}-A={a}"+".txt", "r")
        res_rules_raw[f"CV={i}-A={a}"] = f.read().encode()
        f.close()

Transform the raw output into sets of rules. 

In [ ]:
rules_per_cv = parse_amie(res_rules_raw)

Retrieve the indexes for each CV.

In [ ]:
indexes = retrieve_indexes(root, cv)
df = pd.read_csv(root+"dfSave.csv")

# 5° Precision

In [ ]:
%%time

rules_per_para = run_precision(root, indexes, rules_per_cv)

### Add parameters

Describe the combinaison of these new parameters.

In [ ]:
new_parameters = []
for minc in minc_LIST:
    for minhc in minhc_LIST:
        new_parameters.append([("stdConfidence", minc), ("headCoverage", minhc)])
print(new_parameters)

Create the DataFrames corresponding to these parameters.

In [ ]:
new_dfs = add_parameters(rules_per_para, new_parameters)

Print the precision for each parameters.

In [ ]:
fig, _axs = plt.subplots(nrows=1, ncols=1, figsize=(30,10)) 
to_print, to_print_std = mean_and_std_precision(new_dfs, cv)

to_print.plot.bar(ax=_axs, rot=0, yerr=to_print_std, ylim=[0,1])

# 6° Prediction

In [ ]:
to_print_save =  []
to_print_std_save = []

In [ ]:
%%time

# prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, democracy_proportional, indexes, True)
prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, democracy, indexes, True)

accuracy_prediction = compute_accuracy_prediction(prediction_per_rules_raw, df, indexes)

to_print, to_print_std = mean_and_std_vote(accuracy_prediction, "Demo", cv)
to_print_save.append(to_print)
to_print_std_save.append(to_print_std)

max_possible = bestScorePossible(prediction_per_rules_count, cv)
print_accuracy(to_print_save[-1], to_print_std_save[-1], max_possible, colors_to_print)

In [ ]:
%%time

# prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, democracy, indexes, True)
prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, safe, indexes, True)

accuracy_prediction = compute_accuracy_prediction(prediction_per_rules_raw, df, indexes)

to_print, to_print_std = mean_and_std_vote(accuracy_prediction, "Safe", cv)
to_print, to_print_std = merge(to_print_save[-1], to_print, to_print_std_save[-1], to_print_std)
to_print_save.append(to_print)
to_print_std_save.append(to_print_std)

print_accuracy(to_print_save[-1], to_print_std_save[-1], max_possible, colors_to_print)

In [ ]:
%%time

prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, expert, indexes, True, ranking=(["precision_train", "precision_test"], [False, False]))

accuracy_prediction = compute_accuracy_prediction(prediction_per_rules_raw, df, indexes)

to_print, to_print_std = mean_and_std_vote(accuracy_prediction, "expert-precision", cv)
to_print, to_print_std = merge(to_print_save[-1], to_print, to_print_std_save[-1], to_print_std)
to_print_save.append(to_print)
to_print_std_save.append(to_print_std)

print_accuracy(to_print_save[-1], to_print_std_save[-1], max_possible, colors_to_print)

In [ ]:
%%time

prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, expert, indexes, True, ranking=(["headCoverage", "precision_train", "precision_test"], [True, False, False]))

accuracy_prediction = compute_accuracy_prediction(prediction_per_rules_raw, df, indexes)

to_print, to_print_std = mean_and_std_vote(accuracy_prediction, "expert-coverage", cv)
to_print, to_print_std = merge(to_print_save[-1], to_print, to_print_std_save[-1], to_print_std)
to_print_save.append(to_print)
to_print_std_save.append(to_print_std)

print_accuracy(to_print_save[-1], to_print_std_save[-1], max_possible, colors_to_print)

In [ ]:
%%time

prediction_per_rules_count, prediction_per_rules_raw = prediction_test(root, cv, new_dfs, expert, indexes, True, ranking=(["size_hypothese", "precision_train", "precision_test"], [False, False, False]))

accuracy_prediction = compute_accuracy_prediction(prediction_per_rules_raw, df, indexes)

to_print, to_print_std = mean_and_std_vote(accuracy_prediction, "expert-length", cv)
to_print, to_print_std = merge(to_print_save[-1], to_print, to_print_std_save[-1], to_print_std)
to_print_save.append(to_print)
to_print_std_save.append(to_print_std)

print_accuracy(to_print_save[-1], to_print_std_save[-1], max_possible, colors_to_print)

In [38]:
new_dfs['CV=0-A=2\nstdC=0\nhC=0'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, ?a income-GTE-25 True => ?a approval False to ?a creditScore-GTE-50 False => ?a approval False
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   hypotheses       76 non-null     object 
 1   conclusion       76 non-null     object 
 2   size_hypothese   76 non-null     int64  
 3   headCoverage     76 non-null     float64
 4   stdConfidence    76 non-null     float64
 5   pcaConfidence    76 non-null     float64
 6   precision_train  76 non-null     float64
 7   precision_test   76 non-null     float64
dtypes: float64(5), int64(1), object(2)
memory usage: 5.3+ KB


In [39]:
new_dfs['CV=0-A=2\nstdC=0\nhC=0'].describe()

,size_hypothese,headCoverage,stdConfidence,pcaConfidence,precision_train,precision_test
count,76.0,76.000000,76.000000,76.000000,76.000000,76.000000
mean,1.0,0.340588,0.562019,0.562019,0.546444,0.547244
std,0.0,0.229879,0.271337,0.271337,0.203413,0.204411
min,1.0,0.052636,0.101281,0.101281,0.010155,0.011100
25%,1.0,0.160832,0.292190,0.292190,0.426574,0.429321
50%,1.0,0.223525,0.634398,0.634398,0.577086,0.574309
75%,1.0,0.575770,0.780373,0.780373,0.686182,0.688396
max,1.0,0.871239,0.999903,0.999903,0.871239,0.875412


In [40]:
new_dfs['CV=0-A=2\nstdC=0\nhC=0']

,hypotheses,conclusion,size_hypothese,headCoverage,stdConfidence,pcaConfidence,precision_train,precision_test
?a income-GTE-25 True => ?a approval False,"(?a income-GTE-25 True,)",?a approval False,1,0.544175,0.725691,0.725691,0.441589,0.438561
?a income-GTE-75 True => ?a approval False,"(?a income-GTE-75 True,)",?a approval False,1,0.159032,0.632880,0.632880,0.713287,0.714406
?a loanAmount-LTE-820905.0 ?b => ?a approval ?b,"(?a loanAmount-LTE-820905.0 ?b,)",?a approval ?b,1,0.783743,0.783743,0.783743,0.783743,0.787244
?a rate-GTE-75 True => ?a approval False,"(?a rate-GTE-75 True,)",?a approval False,1,0.196787,0.788970,0.788970,0.635916,0.635147
?a creditScore-GTE-75 ?b => ?a approval ?b,"(?a creditScore-GTE-75 ?b,)",?a approval ?b,1,0.639471,0.639471,0.639471,0.639471,0.644657
...,...,...,...,...,...,...,...,...
?a income/yearlyReimbursement-LTE-3.33 False => ?a approval True,"(?a income/yearlyReimbursement-LTE-3.33 False,)",?a approval True,1,0.219857,0.956162,0.956162,0.010155,0.011100
?a creditScore-GTE-50 False => ?a approval True,"(?a creditScore-GTE-50 False,)",?a approval True,1,0.109816,0.220320,0.220320,0.498740,0.502400
?a monthDuration-GTE-25 False => ?a approval False,"(?a monthDuration-GTE-25 False,)",?a approval False,1,0.227192,0.917050,0.917050,0.426574,0.429321
?a creditScore-GTE-25 False => ?a approval False,"(?a creditScore-GTE-25 False,)",?a approval False,1,0.193982,0.779365,0.779365,0.358999,0.359273
